In [1]:
import json 
import pandas as pd
import numpy as np
import torch
import itertools
from collections import defaultdict, Counter
%matplotlib inline
import matplotlib.pyplot as plt

import datasets
import qanom
from qanom.annotations.common import read_annot_csv


## Scoring QA-level Condfidence

In [2]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
model = AutoModelForSeq2SeqLM.from_pretrained("kleinay/qanom-seq2seq-model-joint")
tokenizer = AutoTokenizer.from_pretrained("kleinay/qanom-seq2seq-model-joint")

In [3]:
input_seq = "parse: yesterday , you<extra_id_10> took<extra_id_10> my hand with your fork to ask me out .<extra_id_1> took"
input_ids = tokenizer(input_seq, return_tensors="pt").input_ids
decoder_input_ids = tokenizer("<pad>", add_special_tokens=False, return_tensors="pt").input_ids
# decoder_input_ids = tokenizer("<pad> when did someone take something _ _?<extra_id_7> yesterday</s>",  
#                               add_special_tokens=False, return_tensors="pt").input_ids

In [21]:
model.branching_strategy = "standard_beam_search"
outputs = model.generate(input_ids, decoder_input_ids=decoder_input_ids, 
                         max_length=100,#len(decoder_input_ids[0]), 
                         output_scores=True,
                         num_beams=3,
                         num_return_sequences=3,
                         return_dict_in_generate=True)
# decoded_output = tokenizer.decode(outputs.sequences[2])
# print(decoded_output)
# decoded_output = tokenizer.decode(outputs.sequences[1])
# print(decoded_output)
decoded_output = tokenizer.decode(outputs.sequences[1])
print(decoded_output)

<pad> when did someone take something _ _?<extra_id_7> yesterday<extra_id_9> who _ _ took something _ _?<extra_id_7> you<extra_id_9> what did someone take _ _ _?<extra_id_7> my hand with your fork<extra_id_9> why did someone take something _ _?<extra_id_7> to ask me out</s><pad><pad><pad><pad><pad><pad><pad>


In [30]:
print(outputs.sequences_scores)
print(outputs.sequences_scores.exp())

tensor([-0.0379, -0.0514, -0.0531])
tensor([0.9628, 0.9499, 0.9483])


In [ ]:
# Note: we only get `scores` for tokens that was generated during this decoidng, substracting tokens 
# that were given as input by `decoder_input_ids`. This can explain why "<pad>" is always not included.  
scores = torch.stack(outputs.scores)
scores = scores.transpose(0,1)
scores.shape

torch.Size([3, 74, 32101])

In [ ]:
torch.stack(outputs.beam_indices[0]).shape

torch.Size([74])

In [9]:
# we should use this new functions to get transition probabilities:
trs_bs = model.compute_transition_beam_scores(
    sequences=outputs.sequences,
    scores=outputs.scores, 
    beam_indices=outputs.beam_indices
)
# Following https://github.com/huggingface/transformers/issues/15869
print("Summ:", torch.sum(trs_bs, dim=1), "Expected:", outputs.sequences_scores)
print("Sum/length:", torch.sum(trs_bs, dim=1)/len(outputs.beam_indices[0]), "Expected:", outputs.sequences_scores)


RuntimeError: The size of tensor a (6) must match the size of tensor b (74) at non-singleton dimension 1

In [ ]:
len(outputs.beam_indices[0])

63

## Use Manual Posterior Computation

We have implemented a computation of "sum of log probabilites" in `QASRLSeq2SeqModel.get_sequence_score`.

Let's test whether it is similar to the sequence_score returned from generate.


In [33]:
from transformers import AutoConfig
import sys, importlib
# sys.path.append("..")
import seq2seq_model
importlib.reload(seq2seq_model)
from seq2seq_model import QASRLSeq2SeqModel

model_name_or_path = "kleinay/qanom-seq2seq-model-joint"
config = AutoConfig.from_pretrained(model_name_or_path)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
model = QASRLSeq2SeqModel.from_pretrained(model_name_or_path,config=config)
# generated = model.generate(input_ids, max_length=120, return_dict_in_generate=True)
output_ids = torch.tensor(outputs.sequences)
print(model.get_sequence_score(input_ids, output_ids[0][1:]))
print(model.get_sequence_score(input_ids, output_ids[1][1:]))
print(model.get_sequence_score(input_ids, output_ids[2][1:]))

/home/nlp/kleinay/tmp/ipykernel_38452/1711341721.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  output_ids = torch.tensor(outputs.sequences)


tensor(0.9717)

In [34]:
print(model.get_sequence_score(input_ids, output_ids[0][1:]))
print(model.get_sequence_score(input_ids, output_ids[1][1:]))
print(model.get_sequence_score(input_ids, output_ids[2][1:]))

tensor(0.9717)
tensor(0.9618)
tensor(0.9594)


In [26]:
output_ids

tensor([[    0,   116,   410,   841,   240,   424,     3,   834,     3,   834,
             3,    58, 32092,  4981, 32090,   113,     3,   834,     3,   834,
           808,   424,     3,   834,     3,   834,     3,    58, 32092,    25,
         32090,   125,   410,   841,   240,     3,   834,     3,   834,     3,
           834,     3,    58, 32092,    82,   609,    28,    39,    21,   157,
         32090,   572,   410,   841,   240,   424,     3,   834,     3,   834,
             3,    58, 32092,    12,   987,   140,    91,     1,     0,     0,
             0,     0,     0,     0,     0],
        [    0,   116,   410,   841,   240,   424,     3,   834,     3,   834,
             3,    58, 32092,  4981, 32090,   113,     3,   834,     3,   834,
           808,   424,     3,   834,     3,   834,     3,    58, 32092,    25,
         32090,   125,   410,   841,   240,     3,   834,     3,   834,     3,
           834,     3,    58, 32092,    82,   609, 32096,    82,   609,    28,
       